<a href="https://colab.research.google.com/github/ebi-ait/hca-ebi-dev-team/blob/feature%2Fdcp-834-programmatic-submissions/scripts/programmatic_submissions/programmatic_submissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmatic Submissions to Ingest

see [ticket 834](https://app.zenhub.com/workspaces/dcp-ingest-product-development-5f71ca62a3cb47326bdc1b5c/issues/ebi-ait/dcp-ingest-central/834)

## Set up  libraries anddependencies

### Install external libraries

In [1]:
!pip install hca-ingest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.0 MB/s 
     |████████████████████████████████| 60 kB 7.3 MB/s 
     |████████████████████████████████| 4.1 MB 36.1 MB/s 
     |████████████████████████████████| 61 kB 7.0 MB/s 
     |████████████████████████████████| 62 kB 302 kB/s 
     |████████████████████████████████| 47 kB 2.1 MB/s 
     |████████████████████████████████| 139 kB 71.5 MB/s 
     |████████████████████████████████| 149 kB 49.3 MB/s 
  Created wheel for hca-ingest: filename=hca_ingest-2.1.0-py3-none-any.whl size=72721 sha256=39b64085b52ef60d9ac818f52664abead24ff6c1cf562f33ec645ffbec427dc2
  Stored in directory: /root/.cache/pip/wheels/96/4f/8c/3974b83843009a0fe32ecc9c98479b37a00c3e7a33e67dcc38
  Created wheel for polling: filename=polling-0.3.2-py3-none-any.whl size=4129 sha256=59cc74c0e58bb07c065852b5c11c86840d45b37edc4abd14da74e18336401034
  Stored in directory: /root/.c

### Load libraries

In [3]:
import requests as rq
import json
from hca_ingest.api.ingestapi import IngestApi

Login and get a token

In [ ]:
# TODO: for now token is obtained from the UI
token = "Bearer "
# code to login

### Set up environment and global variables

In [ ]:
# Environment-related set-up and global variables used across the notebook
accepted_environments = {
    'develop': '.dev',
    'staging': '.staging',
    'production': ''
}

environment = 'staging'  #staging environment by default

# Set up environment value for API's URL
try:
  env_for_url = accepted_environments[environment]
except KeyError:
  print(f"Environment {environment} not recognised. Defaulting to staging")
  env_for_url = accepted_environments['staging']

base_url = f'https://api.ingest{env_for_url}.archive.data.humancellatlas.org'

# Set up API object
api = IngestApi(url=base_url)


## Create a project

This block of code will be dedicated to creating a project within ingest. The following will be assumed:
* A JSON entity is available for use as the "content"

For the purpose of this notebook, everything will be performed in the staging environment. To perform this on other environments (e.g. prod), please update the `environment` variable to any of the values accepted in `accepted_environments`

In [ ]:
# Load the project metadata entity

